# qe_test.ipynb
### Kat Nykiel
## Objectives
- query structure from materials project
- strain structure using pymatgen
- generate QE input files with pymatgen
- submit QE job

### Query structure from materials project (new API)

In [1]:
from mp_api.client import MPRester

/home/knykiel/.conda/envs/cent7/2020.02-py37/atomate2/lib/python3.9/site-packages/pymatgen/analysis/phase_diagram.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Read in new Materials Project API key
with open(os.path.expanduser('~/keys.txt'), 'r') as f:
    key = f.readlines()[0]
    
# Query using new API for material with ID of mp-149 (Si)
with MPRester(key) as m:
    data = m.summary.search(material_ids=["mp-149"])

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
struct = data[0].structure
print(struct)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.866975   3.866975   3.866975
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (2)
  #  SP       a     b     c    magmom
---  ----  ----  ----  ----  --------
  0  Si    0.25  0.25  0.25        -0
  1  Si    0     0     0           -0


### Strain structure using pymatgen

In [4]:
# Check lattice parameters before strain
struct.lattice.abc

(3.8669746532647453, 3.8669746532647453, 3.8669746532647453)

In [5]:
# Apply strain to lattice
struct.apply_strain(0.1)

In [6]:
# Check new lattice vectors
struct.lattice.abc

(4.25367211859122, 4.25367211859122, 4.25367211859122)

### Generate QE input files using pymatgen

In [13]:
from pymatgen.io.pwscf import PWInput
input_set = PWInput(struct, pseudo={'Si':'Si.PBE.UPF'}, control={'calculation':'scf'})
input_set.write_file(filename='Si.in')

In [ ]:
! < Si.in > Si.out

### Extract QE outputs using pymatgen

In [8]:
from pymatgen.io.pwscf import PWOutput
output_set = PWOutput('Si.out')

In [9]:
output_set.final_energy

-63.06668263

TODO: run in nanoHUB with jupyter70 tool
run a standard relaxation
query elastic modulus, get better strain calc
atomic relaxation, then cell relaxation
reproduce convex hull calculations at different pressures
how do we divide this into the most logical pices?
symmetry: make sure pymatgen puts the crystal symmetry in the problem, enforce symmetry from pymatgen detected space group
MgO example (B1, NaCl-> B2, CsCl), transition at 50 GPa
meeting on friday, plan for wednesday lecture

In [12]:
[print(k) for k,v in output_set.data.items()]

lattice_type
ecut
celldm1
celldm2
celldm3
celldm4
celldm5
celldm6
nkpts
energies


[None, None, None, None, None, None, None, None, None, None]

In [11]:
# Extract outputs using monty's regrep method
output_set.read_pattern(patterns={'energy':r"energy\(sigma->0\)\s+=\s+([\d\-.]+)"})